In [7]:
import pandas as pd
from collections import Counter, defaultdict
from us import states
from utils import get_names2abbrs_dict

In [8]:
# pres_df = pd.read_csv(open('poll_data/president_polls.csv'))
pres_df = pd.read_csv('poll_data/president_polls_latest.csv')

/usr/local/Caskroom/miniconda/base/envs/polling/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (6,22,25,29) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [10]:
def clean_pres_polls(in_df):
    df = in_df.copy()
    polls = list()
    names2abbrs = get_names2abbrs_dict()
    column_list = ['question_id', 'poll_id', 'state', 'fte_grade', 'sample_size', 
                   'population', 'population_full', 'methodology', 'start_date', 
                   'end_date', 'internal', 'partisan', 'tracking', 'created_at', 
                   'url', 'answer', 'candidate_name', 'candidate_party', 'pct']
    for p_id in Counter(df['poll_id']):
        for q_id in Counter(df[df['poll_id'] == p_id]['question_id']):
            mask = (df['poll_id'] == p_id) & (df['question_id'] == q_id) & (df['cycle'] == 2020) & (df['stage'] == 'general')
            rows = df[mask]
            rows = df[column_list]
            if set(rows['candidate_name']) == set(['Joseph R. Biden Jr.', 'Donald Trump']) and len(rows) == 2:
                state = list(set(rows['state']))
                assert len(state) == 1
                state_str = str(state[0])
                if state_str != 'nan':
                    state_str = names2abbrs[state_str]
                else:
                    state_str = 'NAT'
                rows['state_po'] = state_str
                polls.extend(rows)
    df = pd.concat(polls)

In [45]:
def get_pres_poll_d_prob(in_df, raw):
    df = in_df.copy()
    results = list()
    names2abbrs = get_names2abbrs_dict()
    for p_id in Counter(df['poll_id']):
        for q_id in Counter(df[df['poll_id'] == p_id]['question_id']):
            mask = (df['poll_id'] == p_id) & (df['question_id'] == q_id) & (df['cycle'] == 2020) & (df['stage'] == 'general')
            rows = df[mask]
            if set(rows['candidate_name']) == set(['Joseph R. Biden Jr.', 'Donald Trump']) and len(rows) == 2:
                state = list(set(rows['state']))
                assert len(state) == 1
                state_str = str(state[0])
                if state_str != 'nan':
                    state_po = names2abbrs[state_str]
                else:
                    state_po = 'NAT'
                    state_str = 'National'
                assert len(set(rows['end_date'])) == 1
                date = rows['end_date'].iloc[0]
                month, day, year = date.split('/')
                month = month.zfill(2)
                day = day.zfill(2)
                year = f'20{year}'
                date = '-'.join([year, month, day])  # get yyyy-mm-dd sortable format
                sample_size = rows['sample_size'].iloc[0].item()
                population = rows['population'].iloc[0]
                
                d_row = rows[rows['candidate_party'] == 'DEM']
                r_row = rows[rows['candidate_party'] == 'REP']
                assert len(d_row) == 1 and len(r_row) == 1
                d_pct = d_row['pct'].item()
                r_pct = r_row['pct'].item()
                if raw:
                    d_prob = d_pct/100
                else:
                    d_prob = d_pct / (d_pct + r_pct)
                result = [date, p_id, q_id, state_str, state_po, sample_size, population, d_prob]
                results.append(result)
    return pd.DataFrame(results, columns=['date', 'poll_id', 'question_id', 'state', 'state_po', 'sample_size', 'population', 'd_prob']).dropna().sort_values(by='date').reset_index(drop=True)

In [35]:
d_probs = get_pres_poll_d_prob(pres_df, raw=False)

In [36]:
d_probs.to_csv('./poll_data/all_pres_polls_d_probs.csv', index=False)

In [47]:
d_probs_raw = get_pres_poll_d_prob(pres_df, raw=True)

In [50]:
d_probs_raw.to_csv('./poll_data/all_pres_polls_d_probs_raw.csv', index=False)